In [1]:
import pandas as pd


from pathlib import Path
import datetime



In [2]:
from update_schema import v2_to_v3, v1_to_v2, reorder_table_to_CDE
from util import read_CDE, read_meta_table, NULL



%load_ext autoreload
%autoreload 2


## 1. get the schemas


## 2. locate the metadata path


## 3. update



## 4.  generate version reports... i.e. changes between versions.

In [3]:
schema_version = "v1"
schema_path = Path.home() / "Projects/ASAP/crn-utils/resource/CDE"
CDEv1 = read_CDE(schema_version, local_path=schema_path)

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v1.csv
read local file


In [4]:
schema_version = "v2.1"
CDEv2 = read_CDE(schema_version, local_path=schema_path)
schema_version = "v3.0"
CDEv3 = read_CDE(schema_version, local_path=schema_path)


metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v2.1.csv
read local file
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
/Users/ergonyc/Projects/ASAP/crn-utils/resource/CDE/ASAP_CDE_v3.0.csv
read local file


> SANITY CHECK: verify reading from google doc works.

In [5]:
CDEv1_ = read_CDE("v1")
CDEv2_ = read_CDE("v2.1")
CDEv3_ = read_CDE("v3.0")

metadata_version: ASAP_CDE_v1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v1
read url
metadata_version: ASAP_CDE_v2.1
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v2.1
read url
metadata_version: ASAP_CDE_v3.0
https://docs.google.com/spreadsheets/d/1c0z5KvRELdT2AtQAH2Dus8kwAyyLrR0CROhKOjpU4Vc/gviz/tq?tqx=out:csv&sheet=v3.0
read url


In [6]:
def compare_CDEs(df1, df2):
    """
    Compares two pandas dataframes and returns if they are identical or not.
    If they are not identical, it returns the differences between the two dataframes.
    """
    
    # Sort both dataframes by 'Table' and 'Field' before comparison
    df1 = df1.sort_values(by=['Table', 'Field']).reset_index(drop=True)
    df2 = df2.sort_values(by=['Table', 'Field']).reset_index(drop=True)
    
    if df1.equals(df2):
        return "The dataframes are identical."
    
    differences = []
    
    # Check for differences in values
    df1_diff = df1 != df2
    diff_rows, diff_cols = df1_diff.stack()[df1_diff.stack()].index.to_list(), df1_diff.columns[df1_diff.any()]
    
    for row, col in diff_rows:
        differences.append(
            f"Difference at row {row}, column '{col}': "
            f"df1 value = {df1.loc[row, col]}, df2 value = {df2.loc[row, col]}"
        )
    
    return differences if differences else "No differences found in data."


In [7]:
compare_CDEs(CDEv1, CDEv1_)

'The dataframes are identical.'

In [8]:
compare_CDEs(CDEv2, CDEv2_)

'The dataframes are identical.'

In [9]:
compare_CDEs(CDEv3, CDEv3_)

["Difference at row 0, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 2, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 5, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 8, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 10, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 11, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 13, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 14, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 16, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 21, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 22, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 25, column 'Validation': df1 value = nan, df2 value = nan",
 "Difference at row 33, column 'Validation':

In [19]:
pd.concat([CDEv3, CDEv3_],axis=1)

,Table,Field,Description,DataType,Required,Validation,Table,Field,Description,DataType,Required,Validation
0,STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-...",STUDY,ASAP_team_name,ASAP Team Name: Name of the ASAP CRN Team. i...,Enum,Required,"[""TEAM-LEE"",""TEAM-HAFLER"",""TEAM-HARDY"", ""TEAM-..."
1,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN,STUDY,ASAP_lab_name,Lab Name. : Lab name that is submitting data...,String,Required,NaN
2,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN,STUDY,project_name,Project Name: A Title of the overall project...,String,Required,NaN
3,STUDY,team_dataset_id,"The ""project_name"" is often too verbose for pr...",String,Required,NaN,STUDY,team_dataset_id,"The ""project_name"" is often too verbose for pr...",String,Required,NaN
4,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN,STUDY,project_dataset,Dataset Name: A unique name is required for ...,String,Required,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
152,CONDITION,protocol_id,ID for referencing the protocols for intervent...,String,Required,NaN,CONDITION,protocol_id,ID for referencing the protocols for intervent...,String,Required,NaN
153,CONDITION,intervention_aux_table,Table which defines all experimental condition...,String,Optional,NaN,CONDITION,intervention_aux_table,Table which defines all experimental condition...,String,Optional,NaN
154,MOUSE,subject_id,The Subject ID. This will be a unique ID for ...,String,Required,NaN,MOUSE,subject_id,The Subject ID. This will be a unique ID for ...,String,Required,NaN
155,MOUSE,strain,Strain: Mouse strain details,String,Required,NaN,MOUSE,strain,Strain: Mouse strain details,String,Required,NaN


In [20]:
from update_schema import filter_table_columns, reorder_table_to_CDE

def v1_to_v2(tables_path: str | Path, out_dir: str|None, CDEv1: pd.DataFrame, CDEv2: pd.DataFrame):
    """
    Load the tables from the tables_path, and update them to the CDEv2 schema.
    Export the new tables to a datestamped out_dir.
    """
    # Get the current date and time
    current_date = datetime.datetime.now()
        
    STUDYv1 = read_meta_table(f"{tables_path}/STUDY.csv")
    PROTOCOLv1 = read_meta_table(f"{tables_path}/PROTOCOL.csv")
    SUBJECTv1 = read_meta_table(f"{tables_path}/SUBJECT.csv")
    CLINPATHv1 = read_meta_table(f"{tables_path}/CLINPATH.csv")
    SAMPLEv1 = read_meta_table(f"{tables_path}/SAMPLE.csv")

    # STUDY
    STUDYv2 = STUDYv1.copy() # don't really need to copy here
    # assert len(SAMPLEv1['preprocessing_references'].unique()) == 1
    STUDYv2['preprocessing_references'] = SAMPLEv1['preprocessing_references'][0]
    STUDYv2['team_dataset_id'] = STUDYv2['project_dataset'].str.replace(" ", "_").str.replace("-", "_")

    # PROTOCOL
    PROTOCOLv2 = PROTOCOLv1.copy()  

    SAMP_CLIN = SAMPLEv1.merge(CLINPATHv1, on="sample_id", how="left")
    SAMP_CLIN['source_sample_id'] = SAMP_CLIN['source_sample_id_x']
    SAMP_CLIN = SAMP_CLIN.drop(columns=['source_sample_id_x','source_sample_id_y'])

    SUBJ_SAMP_CLIN = SUBJECTv1.merge(SAMP_CLIN, on="subject_id", how="left")

    SUBJECTv2 = filter_table_columns(SUBJ_SAMP_CLIN, CDEv2, "SUBJECT")
    CLINPATHv2 = filter_table_columns(SUBJ_SAMP_CLIN, CDEv2, "CLINPATH")
    SAMPLEv2 = filter_table_columns(SUBJ_SAMP_CLIN, CDEv2, "SAMPLE")

    DATAv2 = filter_table_columns(SAMPLEv1, CDEv2, "DATA")

    if out_dir is not None:
        # Prepare output directory
        date_str = current_date.strftime('%Y%m%d')
        export_root = Path(tables_path) / f"{out_dir}_{date_str}"
        export_root.mkdir(parents=True, exist_ok=True)

        # Export the tables
        STUDYv2.to_csv(export_root / "STUDY.csv", index=False)
        PROTOCOLv2.to_csv(export_root / "PROTOCOL.csv", index=False)
        SAMPLEv2.to_csv(export_root / "SAMPLE.csv", index=False)
        SUBJECTv2.to_csv(export_root / "SUBJECT.csv", index=False)
        CLINPATHv2.to_csv(export_root / "CLINPATH.csv", index=False)
        DATAv2.to_csv(export_root / "DATA.csv", index=False)

    return STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2



In [21]:
# load v1 tables 
tables_path = Path.home() / "Projects/ASAP/crn-utils/resource/v1"
STUDYv1 = read_meta_table(f"{tables_path}/STUDY.csv")
PROTOCOLv1 = read_meta_table(f"{tables_path}/PROTOCOL.csv")
SUBJECTv1 = read_meta_table(f"{tables_path}/SUBJECT.csv")
CLINPATHv1 = read_meta_table(f"{tables_path}/CLINPATH.csv")
SAMPLEv1 = read_meta_table(f"{tables_path}/SAMPLE.csv")



In [22]:
STUDYv2, PROTOCOLv2, SAMPLEv2, SUBJECTv2, CLINPATHv2, DATAv2 = v1_to_v2(tables_path, None, CDEv1, CDEv2)

In [23]:
STUDYv2.columns, STUDYv2_.columns

NameError: name 'STUDYv2_' is not defined

In [24]:
set(STUDYv2.columns) - set(STUDYv2_.columns)

NameError: name 'STUDYv2_' is not defined

In [25]:
# load v2 tables for comparison with ouput of update_tables_to_CDEv2_
tables_path = Path.home() / "Projects/ASAP/crn-utils/resource/v2.1"
STUDYv2_ = read_meta_table(f"{tables_path}/STUDY.csv")
PROTOCOLv2_ = read_meta_table(f"{tables_path}/PROTOCOL.csv")
SUBJECTv2_ = read_meta_table(f"{tables_path}/SUBJECT.csv")
CLINPATHv2_ = read_meta_table(f"{tables_path}/CLINPATH.csv")
SAMPLEv2_ = read_meta_table(f"{tables_path}/SAMPLE.csv")
DATAv2_ = read_meta_table(f"{tables_path}/DATA.csv")


## Changes from v2.1 to v3.0

### STUDY Table:
- **New Fields:**
  - `ASAP_team_name` (Required)
  - `number_samples` (Required)
  - `sample_types` (Required)
  - `metadata_tables` (Required)
  - `PI_ORCID` (Optional)

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### SAMPLE Table:
- **New Fields:**
  - `condition_id` (Required)

- **Fields Moved from Another Table:**
  - `subject_id` moved from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - None

---

### SUBJECT Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### CLINPATH Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - `subject_id` moved from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - `AMPPD_id` changed from Required to Optional
  - `GP2_id` changed from Required to Optional
  - `ethnicity` changed from Required to Optional
  - `family_history` changed from Required to Optional
  - `last_diagnosis` changed from Required to Optional

---

### PMDBS Table (New Table):
- **New Fields:**
  - `sample_id` (Required)
  - `brain_region` (Required)
  - `hemisphere` (Optional)
  - `region_level_1` (Required)
  - `region_level_2` (Required)
  - `region_level_3` (Required)

- **Fields Moved from Another Table:**
  - `sample_id` moved from `SAMPLE` and `DATA`
  - `brain_region`, `hemisphere`, `region_level_1`, `region_level_2`, `region_level_3` moved from `SAMPLE`

- **Fields Changed from Required to Optional:**
  - None

---

### CONDITION Table (New Table):
- **New Fields:**
  - `condition_id` (Required)
  - `intervention_id` (Required)
  - `protocol_id` (Required)

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### MOUSE Table (New Table):
- **New Fields:**
  - `subject_id` (Required)

- **Fields Moved from Another Table:**
  - `subject_id` moved from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - None

---

### ASSAY_RNAseq Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - `sample_id`, `tissue`, `RIN`, `molecular_source`, `input_cell_count`, `assay`, `sequencing_end`, `sequencing_length`, `sequencing_instrument` moved from `SAMPLE`
  - `sample_id`, `technology`, `omic` moved from `DATA`

- **Fields Changed from Required to Optional:**
  - None

---

### DATA Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None


In [26]:
# Get the current date and time
current_date = datetime.datetime.now()
    
# STUDYv2 = read_meta_table(f"{tables_path}/STUDY.csv")
# PROTOCOLv2 = read_meta_table(f"{tables_path}/PROTOCOL.csv")
# SUBJECTv2 = read_meta_table(f"{tables_path}/SUBJECT.csv")
# CLINPATHv2 = read_meta_table(f"{tables_path}/CLINPATH.csv")
# SAMPLEv2 = read_meta_table(f"{tables_path}/SAMPLE.csv")
# DATAv2 = read_meta_table(f"{tables_path}/DATA.csv")

# STUDY
STUDYv3 = STUDYv2.copy() # don't really need to copy here
STUDYv3['metadata_tables'] = "STUDY, PROTOCOL, SUBJECT, SAMPLE, DATA, CLINPATH, PMDBS, CONDITION, ASSAY_RNAseq"
# STUDYv3['number_samples'] = STUDYv2['number_of_brain_samples']
# STUDYv3['sample_types'] = STUDYv2['brain_regions']
STUDYv3.rename(columns={"number_of_brain_samples": "number_samples", "brain_regions": "sample_types"}, inplace=True)

# PROTOCOL
PROTOCOLv3 = PROTOCOLv1.copy()  

# SUBJECT
SUBJECTv3 = filter_table_columns(SUBJECTv2, CDEv3, "SUBJECT")


In [27]:

# SAMPLE
SAMPLEv3 = filter_table_columns(SAMPLEv2, CDEv3, "SAMPLE")

# DATA
DATAv3 = filter_table_columns(DATAv2, CDEv3, "DATA")
DATAv3 = pd.merge(DATAv3, SAMPLEv3[['sample_id','time']], on="sample_id", how="left")


In [28]:

# CLINPATH
CLINPATHv3 = filter_table_columns(CLINPATHv2, CDEv3, "CLINPATH")
# move these to CLINPATHv3 from SUBJECTv2
cols_from_SUBJECTv2 = [
    'AMPPD_id',
    'GP2_id',
    'ethnicity',
    'family_history',
    'last_diagnosis',
    'age_at_onset',
    'age_at_diagnosis',
    'first_motor_symptom',
    'hx_dementia_mci',
    'hx_melanoma',
    'education_level',
    'smoking_status',
    'smoking_years',
    'APOE_e4_status',
    'cognitive_status',
    'time_from_baseline']


In [29]:
CLINPATHv3_cols = [col for col in CLINPATHv3.columns if col not in cols_from_SUBJECTv2]
cols_from_SUBJECTv2 += ['subject_id']

In [30]:

CLINPATHv3_ = pd.merge(CLINPATHv3[CLINPATHv3_cols], SUBJECTv2[cols_from_SUBJECTv2], on="subject_id", how="left")


In [31]:

# PMDBS
PMDBSv3 = filter_table_columns(SAMPLEv2, CDEv3, "PMDBS")

# ASSAY_RNAseq
ASSAY_RNAseqv3 = filter_table_columns(SAMPLEv2, CDEv3, "ASSAY_RNAseq")

ASSAY_RNAseqv3['technology'] = DATAv2['technology'][0]
ASSAY_RNAseqv3['omic'] = DATAv2['omic'][0]



In [32]:

# CONDITION
# construct this table

def intervention_typer(x):
    if x is None:
        return "Control"
    else:
        x = x.lower() 
        if x in [ NULL, "", "Control", "Healthy", "HC", "No PD or other neurological disease", "No PD or other neurological disorder"]:
            return "Control"
        elif x in ["PD", "Parkinson's Disease"]:
            return "Case"
        elif x == "Other neurological disorder":
            return "Other Control"
        else:
            return "Case"

CONDITIONv3 = pd.DataFrame(columns=CDEv3[CDEv3['Table'] == "CONDITION"]['Field'])
CONDITIONv3['condition_id'] = SUBJECTv3['primary_diagnosis'].unique()
CONDITIONv3['intervention_name'] = "Case-Control"
CONDITIONv3['intervention_type'] = CONDITIONv3['condition_id'].apply(intervention_typer)

# SAMPLEv3['condition_id'] = SAMPLEv3['sample_id'].map(SUBJECTv3.set_index('subject_id')['primary_diagnosis'])

In [33]:

# Update function for CDEv3 (transforming from v2.1 to v3)
def intervention_typer(x):
    if x is None:
        return "Control"
    else:
        x = x.lower() 
        if x in [ NULL, "", "Control", "Healthy", "HC", "No PD or other neurological disease", "No PD or other neurological disorder"]:
            return "Control"
        elif x in ["PD", "Parkinson's Disease"]:
            return "Case"
        elif x == "Other neurological disorder":
            return "Other Control"
        else:
            return "Case"



In [34]:
            
def v2_to_v3_PMDBS(tables_path: str | Path, out_dir: str, CDEv2: pd.DataFrame, CDEv3: pd.DataFrame):
    """
    Load the tables from the tables_path, and update them to the CDEv3 schema.
    Export the new tables to a datestamped out_dir.
    """
    current_date = datetime.datetime.now()

    # Load the tables
    STUDYv2 = read_meta_table(f"{tables_path}/STUDY.csv")
    PROTOCOLv2 = read_meta_table(f"{tables_path}/PROTOCOL.csv")
    SUBJECTv2 = read_meta_table(f"{tables_path}/SUBJECT.csv")
    CLINPATHv2 = read_meta_table(f"{tables_path}/CLINPATH.csv")
    SAMPLEv2 = read_meta_table(f"{tables_path}/SAMPLE.csv")
    DATAv2 = read_meta_table(f"{tables_path}/DATA.csv")

    # STUDY
    STUDYv3 = STUDYv2.copy() # don't really need to copy here
    STUDYv3['metadata_tables'] = "STUDY, PROTOCOL, SUBJECT, SAMPLE, DATA, CLINPATH, PMDBS, CONDITION, ASSAY_RNAseq"
    # STUDYv3['number_samples'] = STUDYv2['number_of_brain_samples']
    # STUDYv3['sample_types'] = STUDYv2['brain_regions']
    STUDYv3.rename(columns={"number_of_brain_samples": "number_samples", "brain_regions": "sample_types"}, inplace=True)

    STUDYv3 = filter_table_columns(STUDYv2, CDEv3, "STUDY")
    
    # PROTOCOL
    PROTOCOLv3 = PROTOCOLv1.copy()  

    # SUBJECT
    SUBJECTv3 = filter_table_columns(SUBJECTv2, CDEv3, "SUBJECT")

    # PMDBS
    PMDBSv3 = filter_table_columns(SAMPLEv2, CDEv3, "PMDBS")
    # ASSAY_RNAseq
    ASSAY_RNAseqv3 = filter_table_columns(SAMPLEv2, CDEv3, "ASSAY_RNAseq")
    ASSAY_RNAseqv3['technology'] = DATAv2['technology'][0]
    ASSAY_RNAseqv3['omic'] = DATAv2['omic'][0]

    # CLINPATH
    CLINPATHv3 = filter_table_columns(CLINPATHv2, CDEv3, "CLINPATH")
    # move these to CLINPATHv3 from SUBJECTv2
    cols_from_SUBJECTv2 = [
        'AMPPD_id',
        'GP2_id',
        'ethnicity',
        'family_history',
        'last_diagnosis',
        'age_at_onset',
        'age_at_diagnosis',
        'first_motor_symptom',
        'hx_dementia_mci',
        'hx_melanoma',
        'education_level',
        'smoking_status',
        'smoking_years',
        'APOE_e4_status',
        'cognitive_status',
        'time_from_baseline']
    CLINPATHv3_cols = [col for col in CLINPATHv3.columns if col not in cols_from_SUBJECTv2]
    cols_from_SUBJECTv2 += ['subject_id']
    CLINPATHv3_ = pd.merge(CLINPATHv3[CLINPATHv3_cols], SUBJECTv2[cols_from_SUBJECTv2], on="subject_id", how="left")

    # PMDBS
    PMDBSv3 = filter_table_columns(SAMPLEv2, CDEv3, "PMDBS")
    # ASSAY_RNAseq
    ASSAY_RNAseqv3 = filter_table_columns(SAMPLEv2, CDEv3, "ASSAY_RNAseq")
    ASSAY_RNAseqv3['technology'] = DATAv2['technology'][0]
    ASSAY_RNAseqv3['omic'] = DATAv2['omic'][0]


    # DATA
    DATAv3 = filter_table_columns(DATAv2, CDEv3, "DATA")
    DATAv3 = pd.merge(DATAv3, SAMPLEv3[['sample_id','time']], on="sample_id", how="left")



    # CONDITION
    # construct this table.  needs to be checked by hand
    CONDITIONv3 = pd.DataFrame(columns=CDEv3[CDEv3['Table'] == "CONDITION"]['Field'])
    CONDITIONv3['condition_id'] = SUBJECTv3['primary_diagnosis'].unique()
    CONDITIONv3['intervention_name'] = "Case-Control"
    CONDITIONv3['intervention_type'] = CONDITIONv3['condition_id'].apply(intervention_typer)
    if out_dir is not None:

        # Prepare output directory
        date_str = current_date.strftime('%Y%m%d')
        export_root = Path(tables_path) / f"{out_dir}_{date_str}"
        export_root.mkdir(parents=True, exist_ok=True)

        # Export the tables
        STUDYv3.to_csv(export_root / "STUDY.csv", index=False)
        PROTOCOLv3.to_csv(export_root / "PROTOCOL.csv", index=False)
        SAMPLEv3.to_csv(export_root / "SAMPLE.csv", index=False)
        SUBJECTv3.to_csv(export_root / "SUBJECT.csv", index=False)
        CLINPATHv3.to_csv(export_root / "CLINPATH.csv", index=False)
        DATAv3.to_csv(export_root / "DATA.csv", index=False)
        PMDBSv3.to_csv(export_root / "PMDBS.csv", index=False)
        CONDITIONv3.to_csv(export_root / "CONDITION.csv", index=False)
        ASSAY_RNAseqv3.to_csv(export_root / "ASSAY_RNAseq.csv", index=False)

    return STUDYv3, PROTOCOLv3, SAMPLEv3, SUBJECTv3, DATAv3, CLINPATHv3, ASSAY_RNAseqv3, PMDBSv3, CONDITIONv3



In [35]:
ans = v2_to_v3_PMDBS(tables_path, None, CDEv2, CDEv3)

STUDYv3, PROTOCOLv3, SAMPLEv3, SUBJECTv3, DATAv3, CLINPATHv3, ASSAY_RNAseqv3, PMDBSv3, CONDITIONv3 = ans

In [36]:
from validate import validate_table, ReportCollector

Streamlit NOT successfully imported


In [37]:
table_dfs = [STUDYv3, PROTOCOLv3, SUBJECTv3, CLINPATHv3, SAMPLEv3, DATAv3, PMDBSv3, CONDITIONv3, ASSAY_RNAseqv3]
tables = ['STUDY', 'PROTOCOL', 'SUBJECT', 'CLINPATH', 'SAMPLE', 'DATA', 'PMDBS', 'CONDITION', 'ASSAY_RNAseq']
for table,df in zip(tables,table_dfs):

    schema = CDEv3[CDEv3['Table'] == table]

    report = ReportCollector(destination="NA")
    full_table_, report = validate_table(df.copy(), table, schema, report)
    report.print_log()


recoding number_samples as int
All required fields are present in *STUDY* table.
🚨⚠️❗ **4 Fields with empty (NULL) values:**

	- other_funding_source: 1/1 empty rows (REQUIRED)

	- publication_DOI: 1/1 empty rows (REQUIRED)

	- publication_PMID: 1/1 empty rows (REQUIRED)

	- PI_google_scholar_id: 1/1 empty rows (OPTIONAL)
🚨⚠️❗ **1 Fields with invalid entries:**
- _*number_samples*_:  invalid values 💩'None'
    - valid ➡️ int or NULL ('NA')

All required fields are present in *PROTOCOL* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- other_reference: 1/1 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

All required fields are present in *SUBJECT* table.
🚨⚠️❗ **1 Fields with empty (NULL) values:**

	- primary_diagnosis_text: 23/25 empty rows (OPTIONAL)
No invalid entries found in Enum fields.

recoding age_at_onset as int
recoding age_at_diagnosis as int
recoding first_motor_symptom as int
All required fields are present in *CLINPATH* table.
🚨⚠️❗ **27 Fields with em

In [ ]:

def summarize_schema_changes(old_schema: pd.DataFrame, new_schema: pd.DataFrame):
    """
    Summarizes the changes made between two schema versions.
    
    Parameters:
    - old_schema (pd.DataFrame): The old schema DataFrame.
    - new_schema (pd.DataFrame): The new schema DataFrame.
    
    Returns:
    - dict: A summary of the changes.
    """
    summary = {}

    # Group schemas by tables
    old_schema_grouped = old_schema.groupby('Table')
    new_schema_grouped = new_schema.groupby('Table')

    all_tables = set(old_schema['Table'].unique()).union(set(new_schema['Table'].unique()))

    for table in all_tables:
        old_table_schema = old_schema_grouped.get_group(table) if table in old_schema_grouped.groups else pd.DataFrame()
        new_table_schema = new_schema_grouped.get_group(table) if table in new_schema_grouped.groups else pd.DataFrame()

        # Extract fields and required status from each version
        old_fields = set(old_table_schema['Field'].tolist())
        new_fields = set(new_table_schema['Field'].tolist())

        old_required = old_table_schema[old_table_schema['Required'] == 'Required']['Field'].tolist()
        new_required = new_table_schema[new_table_schema['Required'] == 'Required']['Field'].tolist()

        # Summarize the changes for each table
        moved_fields = {
            field for field in old_fields.intersection(new_fields)
            if old_table_schema[old_table_schema['Field'] == field]['Table'].iloc[0]
            != new_table_schema[new_table_schema['Field'] == field]['Table'].iloc[0]
        }

        summary[table] = {
            'added_fields': new_fields - old_fields,
            'deleted_fields': old_fields - new_fields,
            'moved_fields': moved_fields,
            'fields_now_required': set(new_required) - set(old_required),
            'fields_now_optional': set(old_required) - set(new_required)
        }

    return summary


In [ ]:
root_paht = Path.home() / "Projects/ASAP/crn-meta-validate"




# Load schemas as DataFrames (ensure proper file paths)
old_schema_v1 = pd.read_csv('path_to_v1_schema.csv')
new_schema_v2_1 = pd.read_csv('path_to_v2_1_schema.csv')
new_schema_v3 = pd.read_csv('path_to_v3_schema.csv')

# Run the schema comparison
schema_changes_v1_to_v2_1 = summarize_schema_changes(old_schema_v1, new_schema_v2_1)
schema_changes_v2_1_to_v3 = summarize_schema_changes(new_schema_v2_1, new_schema_v3)

# Format the changes into bullet lists
formatted_changes_v1_to_v2_1 = format_schema_changes(schema_changes_v1_to_v2_1)
formatted_changes_v2_1_to_v3 = format_schema_changes(schema_changes_v2_1_to_v3)

# Print the results
print("\nChanges from v1 to v2.1:")
for change in formatted_changes_v1_to_v2_1:
    print(change)

print("\nChanges from v2.1 to v3:")
for change in formatted_changes_v2_1_to_v3:
    print(change)


In [ ]:


# Example usage:
# Assume that 'old_schema_v1' and 'new_schema_v2_1' are already loaded as DataFrames
schema_changes_v1_to_v2_1 = summarize_schema_changes(old_schema_v1, new_schema_v2_1)
schema_changes_v2_1_to_v3 = summarize_schema_changes(new_schema_v2_1, new_schema_v3)

# Output the summarized changes for v1 to v2.1 and v2.1 to v3
print("Changes from v1 to v2.1:")
print(schema_changes_v1_to_v2_1)

print("\nChanges from v2.1 to v3:")
print(schema_changes_v2_1_to_v3)


In [ ]:

def summarize_schema_changes(old_schema: pd.DataFrame, new_schema: pd.DataFrame):
    """
    Summarizes the changes made between two schema versions.
    
    Parameters:
    - old_schema (pd.DataFrame): The old schema DataFrame.
    - new_schema (pd.DataFrame): The new schema DataFrame.
    
    Returns:
    - dict: A summary of the changes.
    """
    summary = {}

    # Group schemas by tables
    old_schema_grouped = old_schema.groupby('Table')
    new_schema_grouped = new_schema.groupby('Table')

    all_tables = set(old_schema['Table'].unique()).union(set(new_schema['Table'].unique()))

    for table in all_tables:
        old_table_schema = old_schema_grouped.get_group(table) if table in old_schema_grouped.groups else pd.DataFrame()
        new_table_schema = new_schema_grouped.get_group(table) if table in new_schema_grouped.groups else pd.DataFrame()

        # Extract fields and required status from each version
        old_fields = set(old_table_schema['Field'].tolist())
        new_fields = set(new_table_schema['Field'].tolist())

        old_required = old_table_schema[old_table_schema['Required'] == 'Required']['Field'].tolist()
        new_required = new_table_schema[new_table_schema['Required'] == 'Required']['Field'].tolist()

        # Summarize the changes for each table
        moved_fields = {
            field for field in old_fields.intersection(new_fields)
            if old_table_schema[old_table_schema['Field'] == field]['Table'].iloc[0]
            != new_table_schema[new_table_schema['Field'] == field]['Table'].iloc[0]
        }

        summary[table] = {
            'added_fields': new_fields - old_fields,
            'deleted_fields': old_fields - new_fields,
            'moved_fields': moved_fields,
            'fields_now_required': set(new_required) - set(old_required),
            'fields_now_optional': set(old_required) - set(new_required)
        }

    return summary


In [ ]:

    # Assume that 'old_schema_v1' and 'new_schema_v2_1' are already loaded as DataFrames
    schema_changes_v1_to_v2_1 = summarize_schema_changes(old_schema_v1, new_schema_v2_1)
    schema_changes_v2_1_to_v3 = summarize_schema_changes(new_schema_v2_1, new_schema_v3)

    # Output the summarized changes for v1 to v2.1 and v2.1 to v3
    print("Changes from v1 to v2.1:")
    print(schema_changes_v1_to_v2_1)

    print("\nChanges from v2.1 to v3:")
    print(schema_changes_v2_1_to_v3)


## Tables in v1:
- STUDY
- PROTOCOL
- SUBJECT
- SAMPLE
- CLINPATH


---

## Tables in v2.1:
- STUDY
- PROTOCOL
- SUBJECT
- SAMPLE
- CLINPATH
- ASSAY_RNAseq
- DATA

---

## Tables in v3.0:
- STUDY
- PROTOCOL
- SUBJECT
- SAMPLE
- CLINPATH
- ASSAY_RNAseq (+ Other)
- DATA
- PMDBS
- CONDITION
- MOUSE


## Changes from v1 to v2.1

### STUDY Table:
- **New Fields:**
  - `team_dataset_id` (Required)
  - `alternate_dataset_id` (Optional)

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### SAMPLE Table:
- **New Fields:**
  - `alternate_sample_id` (Optional)

- **Fields Moved from Another Table:**
  - `brain_region` moved from `SAMPLE` (Required)
  - `hemisphere` moved from `SAMPLE` (Required)
  - `region_level_1` moved from `SAMPLE` (Required)
  - `region_level_2` moved from `SAMPLE` (Required)
  - `region_level_3` moved from `SAMPLE` (Required)

- **Fields Changed from Required to Optional:**
  - None

---

### SUBJECT Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### CLINPATH Table:
- **New Fields:**
  - `subject_id` (Required)
  - `source_subject_id` (Required)

- **Fields Moved from Another Table:**
  - `subject_id` added from `SUBJECT` (Required)
  - `source_subject_id` added from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - None

---

### DATA Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - `sample_id` moved from `SAMPLE` (Required)

- **Fields Changed from Required to Optional:**
  - None


## Changes from v2.1 to v3.0

### STUDY Table:
- **New Fields:**
  - `ASAP_team_name` (Required)
  - `number_samples` (Required)
  - `sample_types` (Required)
  - `metadata_tables` (Required)
  - `PI_ORCID` (Optional)

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### SAMPLE Table:
- **New Fields:**
  - `condition_id` (Required)

- **Fields Moved from Another Table:**
  - `subject_id` moved from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - None

---

### SUBJECT Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### CLINPATH Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - `subject_id` moved from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - `AMPPD_id` changed from Required to Optional
  - `GP2_id` changed from Required to Optional
  - `ethnicity` changed from Required to Optional
  - `family_history` changed from Required to Optional
  - `last_diagnosis` changed from Required to Optional

---

### PMDBS Table (New Table):
- **New Fields:**
  - `sample_id` (Required)
  - `brain_region` (Required)
  - `hemisphere` (Optional)
  - `region_level_1` (Required)
  - `region_level_2` (Required)
  - `region_level_3` (Required)

- **Fields Moved from Another Table:**
  - `sample_id` moved from `SAMPLE` and `DATA`
  - `brain_region`, `hemisphere`, `region_level_1`, `region_level_2`, `region_level_3` moved from `SAMPLE`

- **Fields Changed from Required to Optional:**
  - None

---

### CONDITION Table (New Table):
- **New Fields:**
  - `condition_id` (Required)
  - `intervention_id` (Required)
  - `protocol_id` (Required)

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None

---

### MOUSE Table (New Table):
- **New Fields:**
  - `subject_id` (Required)

- **Fields Moved from Another Table:**
  - `subject_id` moved from `SUBJECT` (Required)

- **Fields Changed from Required to Optional:**
  - None

---

### ASSAY_RNAseq Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - `sample_id`, `tissue`, `RIN`, `molecular_source`, `input_cell_count`, `assay`, `sequencing_end`, `sequencing_length`, `sequencing_instrument` moved from `SAMPLE`
  - `sample_id`, `technology`, `omic` moved from `DATA`

- **Fields Changed from Required to Optional:**
  - None

---

### DATA Table:
- **New Fields:**
  - None

- **Fields Moved from Another Table:**
  - None

- **Fields Changed from Required to Optional:**
  - None


## v1: *id Fields by Table

### CLINPATH Table:
- `sample_id`
- `source_sample_id`
- `GP2_id`
- `AMPPD_id`

### SAMPLE Table:
- `source_sample_id`
- `subject_id`
- `organism_ontology_id`

### STUDY Table:
- `ASAP_grant_id`
- `PI_google_scholar_id`

### SUBJECT Table:
- `subject_id`
- `source_subject_id`

---

## v2.1: *id Fields by Table

### CLINPATH Table:
- `subject_id`
- `source_subject_id`

### DATA Table:
- `sample_id`

### SAMPLE Table:
- `sample_id`
- `subject_id`
- `source_sample_id`
- `organism_ontology_id`

### STUDY Table:
- `team_dataset_id`
- `ASAP_grant_id`
- `PI_google_scholar_id`
- `alternate_dataset_id`

### SUBJECT Table:
- `subject_id`
- `source_subject_id`
- `AMPPD_id`
- `GP2_id`

---

## v3.0: *id Fields by Table

### ASSAY_RNAseq Table:
- `sample_id`

### CLINPATH Table:
- `subject_id`
- `source_subject_id`
- `AMPPD_id`
- `GP2_id`

### CONDITION Table:
- `condition_id`
- `intervention_id`
- `protocol_id`

### DATA Table:
- `sample_id`

### MOUSE Table:
- `subject_id`

### PMDBS Table:
- `sample_id`

### SAMPLE Table:
- `sample_id`
- `subject_id`
- `source_sample_id`
- `condition_id`

### STUDY Table:
- `team_dataset_id`
- `ASAP_grant_id`
- `PI_google_scholar_id`
- `alternate_dataset_id`

### SUBJECT Table:
- `subject_id`
- `source_subject_id`
